# Laboratorio 3. Análisis GeoEspacial y Sensores Remotos
- Fabiola Contreras 22787
- María José Villafuerte 22129

## Carga de datos y NVDI

In [ ]:
import rasterio
import numpy as np

# Función para calcular NDVI dado un path, banda NIR y banda Rojo
def calcular_ndvi(ruta, banda_nir, banda_rojo):
    with rasterio.open(ruta) as src:
        nir = src.read(banda_nir).astype("float32")
        rojo = src.read(banda_rojo).astype("float32")
        
        # Evitar división por cero
        ndvi = (nir - rojo) / (nir + rojo + 1e-10)
        
        perfil = src.profile  # Guardamos metadatos para exportar luego
    return ndvi, perfil

# Rutas a los archivos
ruta_2020 = "imagen_2020.tiff"
ruta_2024 = "imagen_2024.tiff"

# Calcular NDVI para cada año (ejemplo: banda Rojo=4, banda NIR=8)
ndvi_2020, perfil = calcular_ndvi(ruta_2020, banda_nir=8, banda_rojo=4)
ndvi_2024, _ = calcular_ndvi(ruta_2024, banda_nir=8, banda_rojo=4)

In [ ]:
ndvi_diff = ndvi_2024 - ndvi_2020

umbral = -0.2
perdida = ndvi_diff < umbral  # Esto es un array booleano

## Visualización

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 4, figsize=(18, 5))

# NDVI 2020
im1 = axs[0].imshow(ndvi_2020, cmap='RdYlGn', vmin=-1, vmax=1)
axs[0].set_title("NDVI 2020")
fig.colorbar(im1, ax=axs[0], fraction=0.046, pad=0.04)

# NDVI 2024
im2 = axs[1].imshow(ndvi_2024, cmap='RdYlGn', vmin=-1, vmax=1)
axs[1].set_title("NDVI 2024")
fig.colorbar(im2, ax=axs[1], fraction=0.046, pad=0.04)

# Diferencia NDVI
im3 = axs[2].imshow(ndvi_diff, cmap='bwr', vmin=-1, vmax=1)
axs[2].set_title("Diferencia NDVI (2024 - 2020)")
fig.colorbar(im3, ax=axs[2], fraction=0.046, pad=0.04)

# Máscara de deforestación
im4 = axs[3].imshow(perdida, cmap='gray')
axs[3].set_title("Máscara Deforestación")
fig.colorbar(im4, ax=axs[3], fraction=0.046, pad=0.04)

for ax in axs:
    ax.axis('off')

plt.tight_layout()
plt.show()


## Área de deforestación

In [ ]:
import rasterio

# Leemos la resolución espacial (tamaño de pixel)
with rasterio.open(ruta_2020) as src:
    resol_x, resol_y = src.res  # tamaño de pixel en unidades del CRS (generalmente metros)

# Área por pixel (en m²)
area_pixel = abs(resol_x * resol_y)

# Contar pixeles con pérdida de vegetación
pixeles_perdida = np.sum(perdida)

# Calcular área total de pérdida
area_perdida_m2 = pixeles_perdida * area_pixel
area_perdida_km2 = area_perdida_m2 / 1e6

# Calcular porcentaje respecto al área total analizada
area_total_m2 = perdida.size * area_pixel
porcentaje_perdida = (area_perdida_m2 / area_total_m2) * 100

print(f"Área pérdida: {area_perdida_km2:.2f} km²")
print(f"Porcentaje pérdida: {porcentaje_perdida:.2f} %")